## Critical Thinking 6
Testing file for CT 6

In [2]:
import tensorflow as tf
import numpy as np
import keras_tuner as kt
from tensorflow import keras
from keras import models, layers

# print(tf.config.list_physical_devices())
# devices = tf.config.list_physical_devices()
# tf.config.set_visible_devices(devices[0], 'CPU')


In [3]:
# Download the CIFAR10 Dataset
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

# Normalize the data
train_images, test_images = train_images/255.0, test_images/255.0

#### CNN Models

In [4]:
# Model from Tensorflow Tutorial
def build_model_tf():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10)) # Ten total output classes

    # Compile model
    model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model

In [5]:
model = build_model_tf()
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 1024)              0

2022-03-19 19:36:17.501063: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Tuned model with pooling
def build_model_pool(hp):
    
    # Tune the number of conv layers
    model = models.Sequential()
    model.add(layers.Input((32,32,3)))

    for i in range(hp.Int("num_conv_layers", 1, 3)):
        
        # Add Conv Layer
        model.add(
            layers.Conv2D(
                # Tune number of units
                hp.Int(f"conv_units_{i}", min_value=32, max_value=128,step=32),
                # Tune kernel size
                kernel_size=hp.Choice(f"kernel_size_{i}", [3,5]),
                # Tune strides
                strides=hp.Int(f"strides_{i}", min_value=1, max_value=3),
                activation='relu'
            ))

        # Add Pooling Layer
        model.add(
            layers.MaxPooling2D((2,2))
        )

    model.add(layers.Flatten())

    if(hp.Boolean("dropout")):
        model.add(layers.Dropout(rate=0.5))

    # Tune number of conv layers
    for i in range(hp.Int("num_dense_layers", 1, 3)):
        model.add(
            layers.Dense(units=hp.Int(f"dense_units_{i}", min_value=32, max_value=256, step=32), activation='relu')
        )

    model.add(layers.Dense(10))

    model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model

In [7]:
# Tuned model w/o pooling
def build_model_no_pool(hp):
    
    # Tune the number of conv layers
    model = models.Sequential()
    model.add(layers.Input((32,32,3)))

    for i in range(hp.Int("num_conv_layers", 1, 3)):
        
        # Add Conv Layer
        model.add(
            layers.Conv2D(
                # Tune number of units
                hp.Int(f"conv_units_{i}", min_value=32, max_value=128,step=32),
                # Tune kernel size
                kernel_size=hp.Choice(f"kernel_size_{i}", [3,5]),
                # Tune strides
                strides=hp.Int(f"strides_{i}", min_value=1, max_value=3),
                activation='relu'
            ))

    model.add(layers.Flatten())

    if(hp.Boolean("dropout")):
        model.add(layers.Dropout(rate=0.5))

    # Tune number of conv layers
    for i in range(hp.Int("num_dense_layers", 1, 3)):
        model.add(
            layers.Dense(units=hp.Int(f"dense_units_{i}", min_value=32, max_value=256, step=32), activation='relu')
        )

    model.add(layers.Dense(10))

    model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model

In [8]:
# Tuned dilated model
def build_model_dilation(hp):
    
    # Tune the number of conv layers
    model = models.Sequential()
    model.add(layers.Input((32,32,3)))

    for i in range(hp.Int("num_conv_layers", 1, 3)):
        
        # Add Conv Layer
        model.add(
            layers.Conv2D(
                # Tune number of units
                hp.Int(f"conv_units_{i}", min_value=32, max_value=128,step=32),
                # Tune kernel size
                kernel_size=hp.Choice(f"kernel_size_{i}", [3,5]),
                # Tune strides
                dilation_rate=hp.Int(f"dilation_{i}", min_value=1, max_value=3),
                activation='relu'
            ))

            # Add Pooling Layer
        model.add(
            layers.MaxPooling2D((2,2))
        )

    model.add(layers.Flatten())

    if(hp.Boolean("dropout")):
        model.add(layers.Dropout(rate=0.5))

    # Tune number of conv layers
    for i in range(hp.Int("num_dense_layers", 1, 3)):
        model.add(
            layers.Dense(units=hp.Int(f"dense_units_{i}", min_value=32, max_value=256, step=32), activation='relu')
        )

    model.add(layers.Dense(10))

    model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model

In [9]:
def build_model_dilation_no_pool(hp):
    
    # Tune the number of conv layers
    model = models.Sequential()
    model.add(layers.Input((32,32,3)))

    for i in range(hp.Int("num_conv_layers", 1, 3)):
        
        # Add Conv Layer
        model.add(
            layers.Conv2D(
                # Tune number of units
                hp.Int(f"conv_units_{i}", min_value=32, max_value=128,step=32),
                # Tune kernel size
                kernel_size=hp.Choice(f"kernel_size_{i}", [3,5]),
                # Tune strides
                dilation_rate=hp.Int(f"dilation_{i}", min_value=1, max_value=3),
                activation='relu'
            ))

    model.add(layers.Flatten())

    if(hp.Boolean("dropout")):
        model.add(layers.Dropout(rate=0.5))

    # Tune number of conv layers
    for i in range(hp.Int("num_dense_layers", 1, 3)):
        model.add(
            layers.Dense(units=hp.Int(f"dense_units_{i}", min_value=32, max_value=256, step=32), activation='relu')
        )

    model.add(layers.Dense(10))

    model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model

### HPO

In [10]:
MAX_TRIALS = 2
EXECUTIONS_PER_TRIAL = 1

tuner_conv_pool = kt.BayesianOptimization(
    hypermodel=build_model_dilation,
    objective='accuracy',
    max_trials=MAX_TRIALS,
    executions_per_trial= EXECUTIONS_PER_TRIAL,
    overwrite=True,
    directory="hpo/pooling",
    project_name="CIFAR10"
)

print(tuner_conv_pool.search_space_summary())

Search space summary
Default search space size: 7
num_conv_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
conv_units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': None}
kernel_size_0 (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dilation_0 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
dropout (Boolean)
{'default': False, 'conditions': []}
num_dense_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
dense_units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
None


In [18]:
tuner_conv_pool.search(train_images, train_labels, epochs=2, batch_size=250, verbose=1)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
num_conv_layers   |1                 |?                 
conv_units_0      |64                |?                 
kernel_size_0     |3                 |?                 
dilation_0        |2                 |?                 
dropout           |True              |?                 
num_dense_layers  |1                 |?                 
dense_units_0     |160               |?                 

Epoch 1/2


In [11]:
best = tuner_conv_pool.get_best_models(1)